# Сервіс управління завданнями "TaskFlow"

## Короткий опис

TaskFlow - це сервіс для управління завданнями та проектами, який дозволяє користувачам створювати завдання, призначати їх іншим користувачам, відстежувати прогрес та генерувати звіти про виконану роботу. Сервіс забезпечує ефективну організацію робочого процесу та підвищує продуктивність команди.

## Основні функції

1. Реєстрація та автентифікація користувачів
2. Створення та управління проектами
3. Додавання та призначення завдань
4. Відстеження статусу завдань
5. Генерація звітів про виконану роботу
6. Сповіщення про зміни статусу завдань

## Діаграма класів

Діаграма класів відображає основні сутності системи та їх взаємозв'язки.

In [2]:
%%writefile class_diagram.puml
@startuml TaskFlow - Діаграма класів

skinparam classAttributeIconSize 0

class User {
  - id: int
  - username: string
  - email: string
  - password: string
  + register(): void
  + login(): boolean
  + updateProfile(): void
}

class Project {
  - id: int
  - name: string
  - description: string
  - startDate: date
  - endDate: date
  + create(): void
  + update(): void
  + delete(): void
  + addMember(user: User): void
}

class Task {
  - id: int
  - title: string
  - description: string
  - status: TaskStatus
  - priority: Priority
  - dueDate: date
  + create(): void
  + update(): void
  + delete(): void
  + assignTo(user: User): void
  + changeStatus(status: TaskStatus): void
}

enum TaskStatus {
  TODO
  IN_PROGRESS
  REVIEW
  DONE
}

enum Priority {
  LOW
  MEDIUM
  HIGH
  CRITICAL
}

class Comment {
  - id: int
  - text: string
  - createdAt: datetime
  + create(): void
  + update(): void
  + delete(): void
}

class Notification {
  - id: int
  - message: string
  - createdAt: datetime
  - isRead: boolean
  + create(): void
  + markAsRead(): void
}

class ReportService {
  + generateTaskReport(project: Project): Report
  + generateUserReport(user: User): Report
}

class Report {
  - id: int
  - type: string
  - generatedAt: datetime
  - content: string
  + download(): void
}

class AuthService {
  + authenticate(email: string, password: string): boolean
  + generateToken(user: User): string
  + validateToken(token: string): boolean
}

' Зв'язки між класами
User "1" -- "*" Project : створює >
User "*" -- "*" Project : є учасником >
User "1" -- "*" Task : призначено >
User "1" -- "*" Comment : створює >
User "1" -- "*" Notification : отримує >

Project "1" *-- "*" Task : містить >

Task "1" *-- "*" Comment : має >

ReportService ..> Report : створює >
ReportService ..> Project : використовує >
ReportService ..> User : використовує >

User ..> AuthService : використовує >

@enduml

Overwriting class_diagram.puml


## Діаграма послідовності

Діаграма послідовності показує процес створення та призначення завдання.

In [3]:
%%writefile sequence_diagram.puml
@startuml TaskFlow - Діаграма послідовності

actor Користувач as User
participant "Веб-інтерфейс" as Frontend
participant "Сервер" as Backend
participant "Сервіс завдань" as TaskService
participant "База даних" as Database
participant "Сервіс сповіщень" as NotificationService

User -> Frontend : Створити нове завдання
activate Frontend

Frontend -> Backend : POST /api/tasks
activate Backend

Backend -> TaskService : createTask(taskData)
activate TaskService

TaskService -> Database : INSERT INTO tasks
activate Database
Database --> TaskService : task_id
deactivate Database

TaskService --> Backend : Створене завдання
deactivate TaskService

Backend -> Backend : Призначити завдання користувачу

Backend -> Database : UPDATE tasks SET assignee_id = user_id
activate Database
Database --> Backend : Успішно оновлено
deactivate Database

Backend -> NotificationService : sendNotification(assignee_id, task_id)
activate NotificationService

NotificationService -> Database : INSERT INTO notifications
activate Database
Database --> NotificationService : notification_id
deactivate Database

NotificationService --> Backend : Сповіщення надіслано
deactivate NotificationService

Backend --> Frontend : Відповідь: Завдання створено та призначено
deactivate Backend

Frontend --> User : Показати повідомлення про успіх
deactivate Frontend

@enduml

Overwriting sequence_diagram.puml


## Діаграма активності

Діаграма активності демонструє процес обробки запиту на генерацію звіту.

In [4]:
%%writefile activity_diagram.puml
@startuml TaskFlow - Діаграма активності

title Процес генерації звіту

start

:Користувач запитує звіт;

if (Користувач авторизований?) then (так)
  :Перевірити права доступу;
  
  if (Має права на звіт?) then (так)
    :Отримати параметри звіту;
    
    fork
      :Зібрати дані про завдання;
    fork again
      :Зібрати дані про користувачів;
    end fork
    
    :Обробити дані;
    
    :Сформувати звіт;
    
    :Зберегти звіт у базі даних;
    
    :Надіслати звіт користувачу;
    
    :Показати повідомлення про успіх;
  else (ні)
    :Показати повідомлення про відсутність прав;
  endif
else (ні)
  :Перенаправити на сторінку авторизації;
  
  :Показати повідомлення про необхідність авторизації;
endif

stop

@enduml

Overwriting activity_diagram.puml


## Висновок

Розроблений сервіс TaskFlow дозволяє ефективно управляти завданнями та проектами. Діаграми класів, послідовності та активності демонструють структуру та процеси роботи сервісу, що допомагає краще зрозуміти його архітектуру та функціональність.

Для візуалізації діаграм можна використовувати онлайн-сервіс PlantUML або розширення для VS Code.